In [2]:
# Import Necessary Packages in Python
import os
import rpy2
import numpy as np
import pandas as pd
import plotly.graph_objs as go

from numpy import ndarray
from pandas import Series, DataFrame
from plotly.graph_objs import Figure

# Activate R Environment in Jupyter
%load_ext rpy2.ipython

C:\Users\Sayak\AppData\Local\Programs\Python\Python311\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [6]:
# Import Necessary Packages in R
%R library(tidyverse)  # R packages for data science
%R library(tximport)  # Package to calculate transcript-level estimates
%R library(ensembldb)  # Utilities to create and use Ensembl-based annotation databases
%R library(EnsDb.Hsapiens.v86)  # Ensembl based annotation package

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: 'BiocGenerics'


R[write to console]: The following objects are masked from 'package:lubridate':

    intersect, setdiff, union


R[write to console]: The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


R[write to console]: The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


R[write to console]: Loading required package: GenomicRanges

R[w

'EnsDb.Hs...,'ensembldb','Annotati...,...,'datasets','methods','base'


In [72]:
# Define file paths to avoid repetitive typing and ensure consistency
aligned_data_path = '..//Aligned-Data//'
figure_path = '..//Figures//'
processed_data_path = '..//Processed-Data//'

In [70]:
# Generate an array of full paths to 'abundance.tsv' produced by kallisto (Required for tximport)
abundance_path_arr = ['D:\Bioinformatics\Sameek-Sen-Lab\Aligned-Data' + '\\' + path + '\\abundance.tsv' for path in os.listdir(aligned_data_path)]
abundance_path_arr

['D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S1\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S10\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S11\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S114\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S12\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S13\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S14\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S15\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S16\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S17\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S2\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S3\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S4\\abundance.tsv',
 'D:\\Bioinformatics\\Sameek-Sen-Lab\\Aligned-Data\\S5

In [76]:
%R -i abundance_path_arr # Import in R environment

In [81]:
# Retrieve matching transcript IDs and gene names from ENSEMBLE Database
%R transcript_gene_annotation_gr <- ensembldb::transcripts(EnsDb.Hsapiens.v86)  # gr = GRanges (Genomic Ranges)
%R transcript_gene_annotation_tibble <- tibble::as_tibble(tx) # convert them to a tibble for easier manipulation

,seqnames,start,end,width,strand,tx_id,gene_name
1,1,11869,14409,2541,1,ENST00000456328,DDX11L1
2,1,12010,13670,1661,1,ENST00000450305,DDX11L1


In [84]:
%R head(transcript_gene_annotation_tibble, 2)

,target_id,gene_name
1,ENST00000456328,DDX11L1
2,ENST00000450305,DDX11L1


In [83]:
# Select only the 'tx_id' and 'gene_name' columns from `transcript_gene_annotation_tibble`
# Rename 'tx_id' to 'target_id' for `tximport()` function
%R transcript_gene_annotation_tibble <- transcript_gene_annotation_tibble %>% dplyr::select('tx_id', 'gene_name') %>% dplyr::rename('target_id' = 'tx_id')

Error in `dplyr::select()`:
! Can't subset columns that don't exist.
✖ Column `tx_id` doesn't exist.
Run `rlang::last_trace()` to see where the error occurred.


R[write to console]: Warning message:

R[write to console]: In cleanColumns(x, unique(c(columns, "tx_id"))) :
R[write to console]: 
 
R[write to console]:  Column 'id' is not present in the database and has been removed




Error in dplyr::select(., "tx_id", "gene_name") :


RInterpreterError: Failed to parse and evaluate line "transcript_gene_annotation_tibble <- transcript_gene_annotation_tibble %>% dplyr::select('tx_id', 'gene_name') %>% dplyr::rename('target_id' = 'tx_id')".
R error message: 'Error in dplyr::select(., "tx_id", "gene_name") :'

In [86]:
%R head(transcript_gene_annotation_tibble, 2)

,target_id,gene_name
1,ENST00000456328,DDX11L1
2,ENST00000450305,DDX11L1


In [93]:
# Import and summarize transcript-level abundance estimates to gene-level counts (TPM) using 'tximport()' function
# `abundance_path_arr`: An array containing the paths to 'abundance.tsv' files generated by Kallisto
# `type`: The type of software used for abundance estimation, in this case, 'kallisto'
# `tx2gene`: A Dataframe or Similar Object that maps transcript IDs ('tx_id') to gene names ('gene_name')
# `txOut`: Whether to output transcript-level estimates; set to FALSE to only output gene-level estimates
# `countsFromAbundance`: Method used for deriving counts; 'lengthScaledTPM' scales TPMs by average transcript length
# `ignoreTxVersion`: Whether to ignore transcript version (ENST00000380152.1, ENST00000380152.2 etc.) when importing; set to TRUE to ignore
%R gene_level_count_list <- tximport(abundance_path_arr, type = 'kallisto', tx2gene = transcript_gene_annotation_tibble, txOut = FALSE, countsFromAbundance = 'lengthScaledTPM', ignoreTxVersion = TRUE)  # list object

R[write to console]: Note: importing `abundance.h5` is typically faster than `abundance.tsv`

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 5 
R[write to console]: 6 
R[write to console]: 7 
R[write to console]: 8 
R[write to console]: 9 
R[write to console]: 10 
R[write to console]: 11 
R[write to console]: 12 
R[write to console]: 13 
R[write to console]: 14 
R[write to console]: 15 
R[write to console]: 16 
R[write to console]: 17 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 35624

R[write to console]: summarizing abundance

R[write to console]: summarizing counts

R[write to console]: summarizing length



o{'abundance': [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.89718000e-02 ... 0.00000000e+00
  6.23936000e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.53679885e+01 1.55319530e+01 1.39153490e+01 ... 2.03605444e+01
  1.43075420e+01 1.32927870e+01]
 [6.06024700e+01 8.45473381e+01 6.78193500e+01 ... 1.12809243e+02
  8.31705300e+01 8.50692200e+01]], 'counts': [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.84726953e-01 ... 0.00000000e+00
  1.74798372e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00

In [100]:
# gene_level_counts$abundance: Holds TPM values for normalized transcript abundance.
# gene_level_counts$counts: Contains raw or adjusted read counts for each gene, essential for DGE.
# gene_level_counts$length: Stores effective lengths of each transcript.
# gene_level_counts$countsFromAbundance: Specifies method ('lengthScaledTPM', in this case) for deriving counts from abundance.
%R print(names(gene_level_count_list))

[1] "abundance"           "counts"              "length"             
[4] "countsFromAbundance"


'abundance','counts','length','countsFro...


In [102]:
# Extract the raw gene counts and TPM counts from gene_level_count_list to a dataframe
%R raw_count_df <- data.frame(gene_level_count_list$counts)
%R tpm_count_df <- data.frame(gene_level_count_list$abundance)

# Export to python environment
%R -o raw_count_df
%R -o tpm_count_df

# Type annotation
raw_count_df: DataFrame
tpm_count_df: DataFrame

In [103]:
# Take a peek at the count matrix
raw_count_df.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17
5S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
A1BG,0.000000,0.000000,0.884727,1.731269,0.878589,0.000000,0.880750,0.000000,0.000000,2511.164944,0.000000,0.000000,0.000000,1.766093,0.00000,1.747984,0.0
A1CF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.738325,0.731650,10.333252,22.455639,0.000000,0.000000,0.000000,1.446909,0.00000,0.000000,0.0
A2M,10.610636,3.039046,1.498629,3.016140,4.491151,5.921437,4.464646,1.483068,3.007374,22464.830996,3.073017,10.520021,4.522441,6.090698,12.17956,0.000000,0.0
A2ML1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,92.122733,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0


In [108]:
# Rename Column Names
raw_count_df.columns = raw_count_df.columns.str.replace('X', 'S')
tpm_count_df.columns = tpm_count_df.columns.str.replace('X', 'S')

In [110]:
raw_count_df.head(5)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17
5S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
A1BG,0.000000,0.000000,0.884727,1.731269,0.878589,0.000000,0.880750,0.000000,0.000000,2511.164944,0.000000,0.000000,0.000000,1.766093,0.00000,1.747984,0.0
A1CF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.738325,0.731650,10.333252,22.455639,0.000000,0.000000,0.000000,1.446909,0.00000,0.000000,0.0
A2M,10.610636,3.039046,1.498629,3.016140,4.491151,5.921437,4.464646,1.483068,3.007374,22464.830996,3.073017,10.520021,4.522441,6.090698,12.17956,0.000000,0.0
A2ML1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,92.122733,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0


In [112]:
# Save as CSV files
raw_count_df.to_csv(path_or_buf=processed_data_path + 'raw_count_matrix.csv', index=True, index_label='Gene_Names')
tpm_count_df.to_csv(path_or_buf=processed_data_path + 'tpm_count_matrix.csv', index=True, index_label='Gene_Names')